In [1]:
!pip install tensorflow
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install pygame

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.

In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
from matplotlib import pyplot as plt
import matplotlib.animation
from IPython.display import display, clear_output
import random
import pygame

In [3]:
code2action = {
    0:np.array([0, 0, 0, 0]),
    1:np.array([0, 0, 1, 0]),
    2:np.array([0, 0, -1, 0]),
    3:np.array([0, 0, 0, 1]),
    4:np.array([0, 0, 0, -1]),
    5:np.array([0, 0, 1, 1]),
    6:np.array([0, 0, 1, -1]),
    7:np.array([0, 0, -1, 1]),
    8:np.array([0, 0, -1, -1])
}

class ShrinkingCircleEnv(Env):
    width = height = d = 10
    time = 60
    terminated_penalty = 1
    window = None
    window_size = 512
    clock = None
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 15}
    
    def __init__(self):
        self.action_space = Discrete(9)
        self.bounds = np.array([self.height, self.width, self.height, self.width]) - 1
        self.observation_space = Box(np.zeros(4), self.bounds)
        
        
        self.state = (np.random.random_sample(size=4)*self.bounds).astype(int)
        
        self.circle_radius = 0
        
        self.timestamp = self.time
        
        # for rendering
        self.log_circles = []
        self.log_hero = []

        
        
    def get_reward(self):
        circle, hero = self.state[:2], self.state[2:]
        return - np.linalg.norm(hero - circle)
        
    def step(self, action):
        new_state = self.state + code2action[action]
        terminated = not ((0 <= new_state).all() and (new_state < self.bounds).all())
        
        self.timestamp -= 1 
        self.truncated = self.timestamp<=0
        
        reward = self.get_reward()
        
        # move center of circle with probability 0.2
        if np.random.binomial(1, 0.2):
            self.state = np.hstack((
                new_state[:2] + random.choice(list(code2action.values()))[2:],
                new_state[2:]
            ))
        
        # to stay on the playing field
        self.state = np.clip(self.state, np.zeros(4), self.bounds)
            
        info = {}
        return self.state, reward, self.truncated, info
    
    def render(self, mode):
        self.render_mode = mode
        circle, hero = self.state[:2], self.state[2:]
        self.log_circles.append(circle)
        self.log_hero.append(hero)
        
        return self._render_frame()
    
    # inspired by https://www.gymlibrary.dev/content/environment_creation/
    def _render_frame(self):
        if self.window is None and self.render_mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode((self.window_size, self.window_size))
        if self.clock is None and self.render_mode == "human":
            self.clock = pygame.time.Clock()
        
        
        circle, hero = self.state[:2], self.state[2:]
        
        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        pix_square_size = (
            self.window_size / self.d
        )  # The size of a single grid square in pixels

        # First we draw the hero
        pygame.draw.rect(
            canvas,
            (255, 0, 0),
            pygame.Rect(
                pix_square_size * hero,
                (pix_square_size, pix_square_size),
            ),
        )
        # Now we draw the circle
        pygame.draw.circle(
            canvas,
            (0, 0, 255),
            (circle + 0.5) * pix_square_size,
            pix_square_size / 3,
        )

        # Finally, add some gridlines
        for x in range(self.d + 1):
            pygame.draw.line(
                canvas,
                0,
                (0, pix_square_size * x),
                (self.window_size, pix_square_size * x),
                width=3,
            )
            pygame.draw.line(
                canvas,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, self.window_size),
                width=3,
            )
        
        font = pygame.font.Font(None, 25)
        text = font.render(f"Time remaining: {str(self.timestamp)}", True,(0, 0, 128))
        text_rect = text.get_rect(center=(self.window_size/6, self.window_size/15))
        
        if self.render_mode == "human":
            # The following line copies our drawings from `canvas` to the visible window
            self.window.blit(canvas, canvas.get_rect())
            self.window.blit(text, text_rect)
            pygame.event.pump()
            pygame.display.update()

            # We need to ensure that human-rendering occurs at the predefined framerate.
            # The following line will automatically add a delay to keep the framerate stable.
            self.clock.tick(self.metadata["render_fps"])
        else:  # rgb_array
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )
        
    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()
    
    def reset(self):
        
        self.state = (np.random.random_sample(size=4)*self.bounds).astype(int)
        self.circle_radius = 0 #min(self.height, self.width)
        self.timestamp = self.time
        
        self.log_circles = []
        self.log_hero = []
        self.window = None
        self.clock = None
        return self.state
    

In [4]:
env = ShrinkingCircleEnv()

/usr/local/lib/python3.9/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


### Let's learn the model

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, CategoryEncoding
from tensorflow.keras.optimizers.legacy import Adam

2023-03-15 16:12:15.038322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
def get_model(states, n_actions=9, compact=False):
    model = Sequential() 
    model.add(keras.Input(shape=(1,states[0])))
    if not compact:
        model.add(Dense(1024, activation='relu'))
        model.add(Dense(100, activation='relu'))
    model.add(Flatten()) 
    model.add(Dense(n_actions, activation='linear'))
    return model

In [7]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [8]:
def get_agent(model, actions):
    policy = EpsGreedyQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [9]:
n_states = env.observation_space.shape
n_actions = env.action_space.n

model = get_model(n_states, n_actions)
dqn = get_agent(model, n_actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])

2023-03-15 16:12:22.279118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-15 16:12:22.287788: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-15 16:12:22.309085: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_2_1/kernel/Assign' id:223 op device:{requested: '', assigned: ''} def:{{{node dense_2_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_2_1/kernel, dense_2_1/kernel/Initializer/stateless_random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes af

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
import time
hist_reward = []
start_time = time.time()
for i in range(3):
    dqn.fit(env, nb_steps=500, visualize=False, verbose=1)
    scores = dqn.test(env, nb_episodes=100, visualize=False)
    mean_reward = np.mean(scores.history['episode_reward'])
    print("--- %s seconds ---" % (time.time() - start_time))
    hist_reward.append([i,time.time() - start_time, mean_reward])

Training for 500 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 12:51 - reward: -4.1231

/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2357: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2023-03-15 16:12:22.791883: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_2/BiasAdd' id:117 op device:{requested: '', assigned: ''} def:{{{node dense_2/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](dense_2/MatMul, dense_2/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-03-15 16:12:22.811623: W tensorflow/c/c_api.cc:291] Operation '{name:'total_3/Assign' id:392 op device:{requested: '', assigned: ''} def:{{{node total_3/Assign}} = AssignVariableOp[_has_manual_control_dependencies=tru

  498/10000 [>.............................] - ETA: 1:08 - reward: -6.7927done, took 3.682 seconds
Testing for 100 episodes ...
Episode 1: reward: -124.725, steps: 60
Episode 2: reward: -273.902, steps: 60
Episode 3: reward: -148.995, steps: 60
Episode 4: reward: -319.936, steps: 60
Episode 5: reward: -260.693, steps: 60
Episode 6: reward: -107.210, steps: 60
Episode 7: reward: -221.728, steps: 60
Episode 8: reward: -183.967, steps: 60
Episode 9: reward: -72.213, steps: 60
Episode 10: reward: -279.119, steps: 60
Episode 11: reward: -254.075, steps: 60
Episode 12: reward: -480.149, steps: 60
Episode 13: reward: -238.604, steps: 60
Episode 14: reward: -358.438, steps: 60
Episode 15: reward: -245.638, steps: 60
Episode 16: reward: -266.338, steps: 60
Episode 17: reward: -209.517, steps: 60
Episode 18: reward: -140.269, steps: 60
Episode 19: reward: -180.319, steps: 60
Episode 20: reward: -320.540, steps: 60
Episode 21: reward: -121.591, steps: 60
Episode 22: reward: -262.915, steps: 60
Ep

In [ ]:
x_qt = [x[1] for x in hist_reward]
y_qt = [x[2] for x in hist_reward]
plt.figure(figsize=(10,6))
for i, x, y in zip(['Deep Q-Learning'], [ x_qt], [ y_qt]):
    
    
    plt.plot(x, y, label=f'Model {i}')
    plt.xlabel('Training time, s')
    plt.ylabel('Validation Reward')
    plt.title('Choosing Architecture')
plt.legend()
plt.grid()